# Anthropic Claude による小さなファイルの文章要約

> *このノートブックは SageMaker Studio の **`Data Science 3.0`** カーネルをご利用ください*

## 概要

この例では、少量のデータ (文字列データ) を (Anthropic Claude モデルを使用して) Amazon Bedrock API に直接取り込み、それぞれのテキストを要約するように指示します。

### Architecture

![](./images/41-text-simple-1.png)

このアーキテクチャでは:

1. 小さなテキスト (もしくは小さなファイル) を読み込む
1. 基盤モデルは入力データを処理する
1. モデルは、取り込んだテキストの概要を含むレスポンスを返す

### ユースケース

このアプローチは、通話記録、会議の記録、書籍、記事、ブログの投稿、およびその他の関連コンテンツを要約するために使用できます。

### 挑戦

このアプローチは、入力テキストまたはファイルがモデルコンテキストの長さに収まる場合に使用できます。ノートブック `02.long-text-summarization-titan.ja.ipynb` では、ユーザーがトークンの制限を超える大きなドキュメントを持っている場合の課題に対処するためのアプローチを探求します。

## セットアップ

このノートブックの残りを実行する前に、以下のセルを実行して (必要なライブラリがインストールされていることを確認し) Bedrock に接続する必要があります。

セットアップの仕組みと ⚠️ **変更が必要か**の詳細については、[Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ja.ipynb) ノートブックを参照してください.


In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet langchain==0.0.309

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ コメントを外して、AWS の設定に応じて以下の行を編集してください  ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

## boto3 で短いテキストの要約
 
Amazon Bedrock への API リクエストの詳細を知るために、このノートブックでは、Boto3 のオペレーションをラップすることで API をシンプルにする langchain に頼るのではなく、API リクエストを作成して Boto3 経由でリクエストを送信する方法を紹介します。

### Boto3 に InvokeModel のリクエスト構文

`InvokeModel` API を使用して基盤モデルにリクエストを送信します。以下は、Anthropic Claude にテキストを送信するための API リクエストの例です。`textGenerationConfig` の推論パラメータは使用するモデルに依存します。Anthropic Claude の推論パラメータは:

- **temperature** は生成のランダム性の度合いを調整する。Temperature が低いほどランダムな生成が少なくなる。
- **top_p** を 1 未満にすると、top_p 以上の確立を持つ最も確立の高いトークンの最小セットのみが保持される。
- **top_k** は生成されたトークンの繰り返しを減らすことができる。値が高いほど、プロンプトまたは以前の生成で出現した回数に比例して、以前に存在したトークンに強いペナルティが適用される。
- **max_tokens_to_sample** は、生成するトークンの最大数。レスポンスが希望する最大文字数まで埋まる保証はない。
- **stop_sequences** は API がそれ以上のトークンの生成を停止するシーケンスである。返されるテキストには停止シーケンスは含まれない。

```python
response = bedrock.invoke_model(body=
                                {"prompt":"this is where you place your input text",
                                 "max_tokens_to_sample":4096,
                                 "temperature":0.5,
                                 "top_k":250,
                                 "top_p":0.5,
                                 "stop_sequences":[]
                                },
                                modelId="anthropic.claude-v2", 
                                accept=accept, 
                                contentType=contentType)

```

### 要約するテキストを含むプロンプトの作成

このノートブックでは、トークンが基盤モデルの最大トークンよりも小さい任意のショートテキストを使用できます。短いテキストの例として、Amazon Bedrock の発表に関する [AWS blog post](https://aws.amazon.com/jp/blogs/machine-learning/announcing-new-tools-for-building-with-generative-ai-on-aws/) の 1 段落を見てみましょう。

プロンプトは `Please provide a summary of the following text.` という指示で始まります。 

In [ ]:
prompt = """
Human: Please provide a summary of the following text.
<text>
AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
for text and images—including Amazons Titan FMs, which consist of two new LLMs we’re also announcing \
today—through a scalable, reliable, and secure AWS managed service. With Bedrock’s serverless experience, \
customers can easily find the right model for what they’re trying to get done, get started quickly, privately \
customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
</text>

Assistant:"""

## プロンプトと推論パラメータを含むリクエストボディの作成

`invoke_model` のリクエスト構文に従って、上記のプロンプトと推論パラメータを使用してリクエストボディを作成します。

In [ ]:
body = json.dumps({"prompt": prompt,
                 "max_tokens_to_sample":4096,
                 "temperature":0.5,
                 "top_k":250,
                 "top_p":0.5,
                 "stop_sequences":[]
                  }) 

## Boto3 経由で基盤モデルを呼び出す

ここでは、リクエストパラメータ `modelId`、`accept`、`contentType` を指定して API リクエストを Amazon Bedrock に送信します。プロンプトに続いて Amazon Bedrock の基盤モデルがテキストを要約します。

In [ ]:
modelId = 'anthropic.claude-v2' # モデルプロバイダーとは異なるバーションを使用する場合は変更してください
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print_ww(response_body.get('completion'))

上記では、Bedrock サービスは特定のプロンプトの要約全体を単一の出力で生成しますが、出力に大量のトークンが含まれている場合、出力が遅くなる可能性があります。

以下では、Bedrock を使用して出力をストリーミングし、ユーザーがモデルによって生成された出力を利用をできるようにする方法について説明します。Bedrock は、`invoke_model_with_response_stream` API をサポートしており、出力をチャンクの形でストリーミングする `ResponseStream` を提供しています。

In [ ]:
response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = list(stream)
output

Bedrock は出力全体を生成する代わりに、モデルから小さなチャンクを送信します。これは消費可能な方法で表示することもできます。

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['completion']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

## まとめ
ここまで、Amazon Bedrock API に直接アクセスできる `boto3` SDK を使って実験してきました。 この API で、Amazon Bedrock に関する AWS ニュースの要約を 2 つの異なる方法で生成するユースケースを見てきました: 全体出力とストリーミング出力生成。

### 発展要素
- このノートブックを作り替えて、Amazon Bedrock から入手できる Amazon Titan や AI21 Labs Jurassic モデルなど、さまざまなモデルを試してみてください。
- プロンプトを特定のユースケースに変更して、さまざまなモデルを試してみてください。
- トークンの長さを試して、サービスのレイテンシーと応答性を理解してください
- さまざまなプロンプトエンジニアリングの原則を適用して、より良い出力を得ます。

## ありがとうございました